# dload yfinance content

In [1]:
import yfinance as yf
import yfinance
from pytickersymbols import PyTickerSymbols
import pandas as pd
end_date="2023-01-10"

In [2]:
stock_data = PyTickerSymbols()
dow30_yahoo = stock_data.get_dow_jones_nyc_yahoo_tickers()
euro50_yahoo = stock_data.get_euro_stoxx_50_frankfurt_yahoo_tickers()
hangseng = [ "2628.HK","2319.HK","2688.HK","0992.HK"
        ,"9988.HK","0101.HK","1810.HK","9618.HK",
        "1044.HK","2331.HK","0883.HK","0017.HK",
        "1398.HK","0002.HK","9633.HK","2269.HK",
        "0267.HK","0003.HK","1093.HK","1038.HK",
        "0027.HK","6690.HK","0241.HK","6098.HK",
        "1109.HK","0012.HK","2020.HK","3690.HK","1209.HK","0669.HK"]


In [3]:
hs = yf.download( 
            tickers = " ".join(dow30_yahoo+euro50_yahoo+hangseng),  #list of tickers
            # period = "10y",         # time period
            end=end_date,
            interval = "1d",       # trading interval
            ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
            prepost = False,
            auto_adjust= True, # adjust for stock splits etc,
            keepna=True)    


[*********************100%***********************]  160 of 160 completed

4 Failed downloads:
- 0VD.F: 1d data not available for startTime=-2208994789 and endTime=1673305200. Only 100 years worth of day granularity data are allowed to be fetched per request.
- NL0013654783.F: No timezone found, symbol may be delisted
- NL00150001Q9.F: No timezone found, symbol may be delisted
- STLA.F: No timezone found, symbol may be delisted


In [4]:
# Downloading data for all stocks
stock_data_tz_ignored = yf.download( 
            tickers = " ".join(dow30_yahoo+euro50_yahoo+hangseng),  #list of tickers
            # period = "10y",         # time period
            end=end_date,
            interval = "1d",       # trading interval
            ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
            prepost = False,
            auto_adjust= True, # adjust for stock splits etc,
            keepna=True)    
stock_data_tz_ignored.columns =  stock_data_tz_ignored.columns.swaplevel(0, 1)


stock_data = yf.download( 
            tickers = " ".join(dow30_yahoo+euro50_yahoo+hangseng),  #list of tickers
            # period = "10y",         # time period
            end=end_date,
            interval = "1d",       # trading interval
            ignore_tz = False,      # ignore timezone when aligning data from different exchanges?
            prepost = False,
            auto_adjust= True, # adjust for stock splits etc,
            keepna=True)
stock_data.columns =  stock_data.columns.swaplevel(0, 1)

[*********************100%***********************]  160 of 160 completed

4 Failed downloads:
- NL00150001Q9.F: No timezone found, symbol may be delisted
- NL0013654783.F: No timezone found, symbol may be delisted
- STLA.F: No timezone found, symbol may be delisted
- 0VD.F: 1d data not available for startTime=-2208994789 and endTime=1673305200. Only 100 years worth of day granularity data are allowed to be fetched per request.
[*********************100%***********************]  160 of 160 completed

4 Failed downloads:
- NL00150001Q9.F: No timezone found, symbol may be delisted
- NL0013654783.F: No timezone found, symbol may be delisted
- STLA.F: No timezone found, symbol may be delisted
- 0VD.F: 1d data not available for startTime=-2208994789 and endTime=1673305200. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [5]:
# Removing data with less than 10 years of records

def filter_data_by_cutoff_date( df_stock:pd.DataFrame, timedelta:str):
    
    latest_date = df_stock.index[-1]
    cutoff_date = latest_date - pd.Timedelta(timedelta)

    # Gathering the rows which have no data prior to cutoff date
    df_stock_before_certain_date = df_stock.loc[ df_stock.index < cutoff_date  ]

    nan_values = df_stock_before_certain_date.isna()
    mask = nan_values.all(axis=0)

    df_stock = df_stock.drop( df_stock.columns[mask], axis=1) 
    
    return df_stock

timedelta ='10Y'
stock_data_tz_ignored = filter_data_by_cutoff_date(stock_data_tz_ignored, timedelta  )
stock_data = filter_data_by_cutoff_date(stock_data, timedelta )

/tmp/ipykernel_3370869/3952319347.py:6: FutureWarning: Units 'M', 'Y' and 'y' do not represent unambiguous timedelta values and will be removed in a future version.
  cutoff_date = latest_date - pd.Timedelta(timedelta)


In [6]:
# Getting data for the currency values of Euro/USD and CNY/USD
eurusd_data = yf.download('EURUSD=X',end=end_date )
cnyusd_data = yf.download('CNYUSD=X',end=end_date )

# Finding latest Date at which all currency date is available
eurusd_d0 = eurusd_data.index[0]
cnyusd_d0 = cnyusd_data.index[0]

latest_date_currency = max(eurusd_d0, eurusd_d0 )

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
latest_date_currency

Timestamp('2003-12-01 00:00:00')

In [7]:
# Only Keep Stock & Currency Data after date for which they are ALL available
eurusd_data = eurusd_data[eurusd_data.index>latest_date_currency ]
cnyusd_data = cnyusd_data[cnyusd_data.index>latest_date_currency ]

stock_data_tz_ignored = stock_data_tz_ignored[stock_data_tz_ignored.index>latest_date_currency]
stock_data_tz_ignored.index = pd.DatetimeIndex( stock_data_tz_ignored.index )

stock_data = stock_data[ stock_data.index.map( lambda x:  x.tz_localize(None) ) > latest_date_currency ]
# NOTE: questionable correctness since tz is assumed to be based on NY time but tz_ignored is general

In [8]:
# Reordering columns such that they are ordered by exhcnage

# creating col index for reordering
ticker_groups = [dow30_yahoo,euro50_yahoo,hangseng]
columns_grouped_by_exchange = [ sorted([ ticker for ticker in stock_data_tz_ignored.columns if ticker[0] in ticker_group]) for ticker_group in  [dow30_yahoo,euro50_yahoo,hangseng]  ]

#reordering
stock_data_tz_ignored = stock_data_tz_ignored[ sum(columns_grouped_by_exchange,[]) ]
stock_data = stock_data[ sum(columns_grouped_by_exchange,[]) ]

idx_exchange_group = [ len(col_group) for col_group in columns_grouped_by_exchange]


In [12]:
stock_data.columns
# idx_exchange_group

MultiIndex([(   'AAPL',  'Close'),
            (   'AAPL',   'High'),
            (   'AAPL',    'Low'),
            (   'AAPL',   'Open'),
            (   'AAPL', 'Volume'),
            (   'AMGN',  'Close'),
            (   'AMGN',   'High'),
            (   'AMGN',    'Low'),
            (   'AMGN',   'Open'),
            (   'AMGN', 'Volume'),
            ...
            ('2628.HK',  'Close'),
            ('2628.HK',   'High'),
            ('2628.HK',    'Low'),
            ('2628.HK',   'Open'),
            ('2628.HK', 'Volume'),
            ('2688.HK',  'Close'),
            ('2688.HK',   'High'),
            ('2688.HK',    'Low'),
            ('2688.HK',   'Open'),
            ('2688.HK', 'Volume')],
           length=620)

In [10]:
# Saving to pickle
import pickle 
import os
import yaml 

dir_ = 'yahoo_data'
os.makedirs(dir_,exist_ok=True)

eurusd_data.columns = pd.MultiIndex.from_tuples( [( 'EURUSD=X' ,col) for col in eurusd_data.columns] )
cnyusd_data.columns = pd.MultiIndex.from_tuples( [( 'CNYUSD=X' ,col) for col in cnyusd_data.columns] )

pickle.dump( pd.concat( [eurusd_data, cnyusd_data], axis=1) , open(os.path.join(dir_,'currencies.pkl'),'wb'))

pickle.dump( stock_data_tz_ignored, open( os.path.join(dir_,'allstock_tz_ignored.pkl'),'wb') )
pickle.dump( stock_data, open(os.path.join(dir_,'allstock.pkl'),'wb') )
pickle.dump( eurusd_data, open(os.path.join(dir_,'eurusd.pkl'),'wb') )
pickle.dump( cnyusd_data, open(os.path.join(dir_,'cnyusd.pkl'),'wb') )



yaml.dump(
    {
        'eurostoxx':[ stock for stock in euro50_yahoo if stock in stock_data_tz_ignored.columns ],
        'dowjones':[ stock for stock in dow30_yahoo if stock in stock_data_tz_ignored.columns ],
        'hangseng':[ stock for stock in hangseng if stock in stock_data_tz_ignored.columns ]
    }, 
    open( os.path.join(dir_,'dict_indicies.yaml'),"w")
)

In [177]:
list(stock_data_tz_ignored.xs('Close',axis=1, level=1).columns )

['0002.HK',
 '0003.HK',
 '0012.HK',
 '0017.HK',
 '0027.HK',
 '0101.HK',
 '0241.HK',
 '0267.HK',
 '0669.HK',
 '0883.HK',
 '0992.HK',
 '1038.HK',
 '1044.HK',
 '1093.HK',
 '1109.HK',
 '1398.HK',
 '1NBA.F',
 '2020.HK',
 '2319.HK',
 '2331.HK',
 '2628.HK',
 '2688.HK',
 '8TI.F',
 'AAPL',
 'AD.AS',
 'ADS.F',
 'ADS1.F',
 'AHOD.F',
 'AHOG.F',
 'AIL.F',
 'ALV.F',
 'AMGN',
 'ASME.F',
 'ASMF.F',
 'ASML.AS',
 'AXAA.F',
 'AXP',
 'BA',
 'BAS.F',
 'BASA.F',
 'BAYN.F',
 'BBVA.F',
 'BCO.F',
 'BMW.F',
 'BMW3.F',
 'BMWB.F',
 'BNP.F',
 'BNPH.F',
 'BOY.F',
 'BSD2.F',
 'BSDK.F',
 'BSN.F',
 'CAT',
 'CRG.F',
 'CRM',
 'CSCO',
 'CVX',
 'DB1.F',
 'DIS',
 'DPW.F',
 'DPWA.F',
 'DTE.F',
 'DTEA.F',
 'ENI.F',
 'ENI1.F',
 'ENL.F',
 'ENLA.F',
 'ESL.F',
 'GS',
 'HD',
 'HMI.F',
 'HON',
 'IBE1.F',
 'IBM',
 'IES.F',
 'IFX.F',
 'IFXA.F',
 'INGA.AS',
 'INN1.F',
 'INNA.F',
 'INTC',
 'ITKA.F',
 'IXD1.F',
 'JNJ',
 'JPM',
 'KC4.F',
 'KO',
 'LIN.F',
 'LOR.F',
 'MBG.F',
 'MCD',
 'MMM',
 'MOH.F',
 'MRK',
 'MSFT',
 'MUV2.F',
 'NKE',
 

## Experimenting

In [13]:
li_start_end_idxs


NameError: name 'li_start_end_idxs' is not defined

In [167]:
li_start_end_idxs

[{'s_idx': 311, 'len': 3},
 {'s_idx': 343, 'len': 2},
 {'s_idx': 350, 'len': 1},
 {'s_idx': 369, 'len': 1},
 {'s_idx': 379, 'len': 1},
 {'s_idx': 413, 'len': 1},
 {'s_idx': 469, 'len': 1},
 {'s_idx': 485, 'len': 1},
 {'s_idx': 539, 'len': 2},
 {'s_idx': 544, 'len': 1},
 {'s_idx': 564, 'len': 2},
 {'s_idx': 611, 'len': 1},
 {'s_idx': 618, 'len': 2},
 {'s_idx': 629, 'len': 1},
 {'s_idx': 633, 'len': 1},
 {'s_idx': 651, 'len': 1},
 {'s_idx': 739, 'len': 1},
 {'s_idx': 759, 'len': 1},
 {'s_idx': 799, 'len': 1},
 {'s_idx': 803, 'len': 1},
 {'s_idx': 838, 'len': 2},
 {'s_idx': 871, 'len': 2},
 {'s_idx': 888, 'len': 1},
 {'s_idx': 905, 'len': 1},
 {'s_idx': 923, 'len': 1},
 {'s_idx': 932, 'len': 1},
 {'s_idx': 994, 'len': 1},
 {'s_idx': 997, 'len': 1},
 {'s_idx': 1011, 'len': 1},
 {'s_idx': 1058, 'len': 1},
 {'s_idx': 1088, 'len': 2},
 {'s_idx': 1119, 'len': 1},
 {'s_idx': 1128, 'len': 1},
 {'s_idx': 1147, 'len': 1},
 {'s_idx': 1154, 'len': 1},
 {'s_idx': 1174, 'len': 1},
 {'s_idx': 1190, 'le

In [14]:
b = [{'s_idx': 311, 'len': 3},
 {'s_idx': 343, 'len': 2},
 {'s_idx': 350, 'len': 1},
 {'s_idx': 369, 'len': 1},
 {'s_idx': 379, 'len': 1},
 {'s_idx': 413, 'len': 1},
 {'s_idx': 469, 'len': 1},
 {'s_idx': 485, 'len': 1},
 {'s_idx': 539, 'len': 2},
 {'s_idx': 544, 'len': 1},
 {'s_idx': 564, 'len': 2},
 {'s_idx': 611, 'len': 1},
 {'s_idx': 618, 'len': 2},
 {'s_idx': 629, 'len': 1},
 {'s_idx': 633, 'len': 1},
 {'s_idx': 651, 'len': 1},
 {'s_idx': 739, 'len': 1},
 {'s_idx': 759, 'len': 1},
 {'s_idx': 799, 'len': 1},
 {'s_idx': 803, 'len': 1},
 {'s_idx': 838, 'len': 2},
 {'s_idx': 871, 'len': 2},
 {'s_idx': 888, 'len': 1},
 {'s_idx': 905, 'len': 1},
 {'s_idx': 923, 'len': 1},
 {'s_idx': 932, 'len': 1},
 {'s_idx': 994, 'len': 1},
 {'s_idx': 997, 'len': 1},
 {'s_idx': 1011, 'len': 1},
 {'s_idx': 1058, 'len': 1},
 {'s_idx': 1088, 'len': 2},
 {'s_idx': 1119, 'len': 1},
 {'s_idx': 1128, 'len': 1},
 {'s_idx': 1147, 'len': 1},
 {'s_idx': 1154, 'len': 1},
 {'s_idx': 1174, 'len': 1},
 {'s_idx': 1190, 'len': 1},
 {'s_idx': 1244, 'len': 1},
 {'s_idx': 1256, 'len': 1},
 {'s_idx': 1260, 'len': 1},
 {'s_idx': 1317, 'len': 1},
 {'s_idx': 1337, 'len': 3},
 {'s_idx': 1391, 'len': 1},
 {'s_idx': 1405, 'len': 1},
 {'s_idx': 1424, 'len': 1},
 {'s_idx': 1448, 'len': 1},
 {'s_idx': 1514, 'len': 1},
 {'s_idx': 1531, 'len': 1},
 {'s_idx': 1609, 'len': 2},
 {'s_idx': 1643, 'len': 2},
 {'s_idx': 1677, 'len': 1},
 {'s_idx': 1695, 'len': 1},
 {'s_idx': 1706, 'len': 1},
 {'s_idx': 1766, 'len': 1},
 {'s_idx': 1772, 'len': 1},
 {'s_idx': 1833, 'len': 1},
 {'s_idx': 1861, 'len': 2},
 {'s_idx': 1904, 'len': 1},
 {'s_idx': 1917, 'len': 1},
 {'s_idx': 1922, 'len': 1},
 {'s_idx': 1928, 'len': 1},
 {'s_idx': 1947, 'len': 1},
 {'s_idx': 1966, 'len': 1},
 {'s_idx': 2018, 'len': 1},
 {'s_idx': 2030, 'len': 1},
 {'s_idx': 2034, 'len': 1},
 {'s_idx': 2092, 'len': 2},
 {'s_idx': 2097, 'len': 2},
 {'s_idx': 2113, 'len': 3},
 {'s_idx': 2153, 'len': 1},
 {'s_idx': 2165, 'len': 1},
 {'s_idx': 2167, 'len': 1},
 {'s_idx': 2183, 'len': 1},
 {'s_idx': 2227, 'len': 1},
 {'s_idx': 2292, 'len': 2},
 {'s_idx': 2308, 'len': 1},
 {'s_idx': 2353, 'len': 1},
 {'s_idx': 2385, 'len': 3},
 {'s_idx': 2419, 'len': 1},
 {'s_idx': 2422, 'len': 1},
 {'s_idx': 2441, 'len': 1},
 {'s_idx': 2453, 'len': 1},
 {'s_idx': 2471, 'len': 1},
 {'s_idx': 2484, 'len': 1},
 {'s_idx': 2543, 'len': 1},
 {'s_idx': 2550, 'len': 1},
 {'s_idx': 2559, 'len': 1},
 {'s_idx': 2611, 'len': 1},
 {'s_idx': 2636, 'len': 2},
 {'s_idx': 2691, 'len': 1},
 {'s_idx': 2699, 'len': 1},
 {'s_idx': 2702, 'len': 1},
 {'s_idx': 2721, 'len': 1},
 {'s_idx': 2742, 'len': 1},
 {'s_idx': 2792, 'len': 1},
 {'s_idx': 2808, 'len': 2},
 {'s_idx': 2869, 'len': 1},
 {'s_idx': 2907, 'len': 2},
 {'s_idx': 2938, 'len': 2},
 {'s_idx': 2957, 'len': 1},
 {'s_idx': 2973, 'len': 1},
 {'s_idx': 3000, 'len': 1},
 {'s_idx': 3046, 'len': 1},
 {'s_idx': 3063, 'len': 1},
 {'s_idx': 3066, 'len': 1},
 {'s_idx': 3080, 'len': 1},
 {'s_idx': 3156, 'len': 3},
 {'s_idx': 3190, 'len': 1},
 {'s_idx': 3195, 'len': 1},
 {'s_idx': 3215, 'len': 1},
 {'s_idx': 3243, 'len': 1},
 {'s_idx': 3259, 'len': 1},
 {'s_idx': 3314, 'len': 1},
 {'s_idx': 3330, 'len': 1},
 {'s_idx': 3385, 'len': 1},
 {'s_idx': 3389, 'len': 1},
 {'s_idx': 3409, 'len': 2},
 {'s_idx': 3455, 'len': 1},
 {'s_idx': 3463, 'len': 1},
 {'s_idx': 3473, 'len': 1},
 {'s_idx': 3475, 'len': 1},
 {'s_idx': 3494, 'len': 1},
 {'s_idx': 3583, 'len': 1},
 {'s_idx': 3586, 'len': 1},
 {'s_idx': 3643, 'len': 1},
 {'s_idx': 3680, 'len': 2},
 {'s_idx': 3710, 'len': 1},
 {'s_idx': 3713, 'len': 1},
 {'s_idx': 3731, 'len': 1},
 {'s_idx': 3746, 'len': 1},
 {'s_idx': 3765, 'len': 1},
 {'s_idx': 3775, 'len': 1},
 {'s_idx': 3836, 'len': 1},
 {'s_idx': 3840, 'len': 1},
 {'s_idx': 3852, 'len': 1},
 {'s_idx': 3901, 'len': 1},
 {'s_idx': 3929, 'len': 3},
 {'s_idx': 3972, 'len': 1},
 {'s_idx': 3982, 'len': 1},
 {'s_idx': 3989, 'len': 1},
 {'s_idx': 3997, 'len': 1},
 {'s_idx': 4016, 'len': 1},
 {'s_idx': 4032, 'len': 1},
 {'s_idx': 4098, 'len': 1},
 {'s_idx': 4102, 'len': 1},
 {'s_idx': 4159, 'len': 1},
 {'s_idx': 4180, 'len': 2},
 {'s_idx': 4234, 'len': 1},
 {'s_idx': 4247, 'len': 2},
 {'s_idx': 4287, 'len': 1},
 {'s_idx': 4291, 'len': 1},
 {'s_idx': 4357, 'len': 2},
 {'s_idx': 4365, 'len': 1},
 {'s_idx': 4374, 'len': 1},
 {'s_idx': 4451, 'len': 2},
 {'s_idx': 4486, 'len': 2},
 {'s_idx': 4518, 'len': 1},
 {'s_idx': 4536, 'len': 1},
 {'s_idx': 4549, 'len': 1},
 {'s_idx': 4608, 'len': 1},
 {'s_idx': 4615, 'len': 1},
 {'s_idx': 4623, 'len': 2},
 {'s_idx': 4676, 'len': 1},
 {'s_idx': 4702, 'len': 3},
 {'s_idx': 4747, 'len': 1},
 {'s_idx': 4755, 'len': 1},
 {'s_idx': 4765, 'len': 1},
 {'s_idx': 4770, 'len': 1},
 {'s_idx': 4789, 'len': 1},
 {'s_idx': 4809, 'len': 1},
 {'s_idx': 4860, 'len': 1},
 {'s_idx': 4876, 'len': 1},
 {'s_idx': 4935, 'len': 1},
 {'s_idx': 4939, 'len': 1}]

In [15]:
sample_lens = [d['len'] for d in b]

In [16]:
from collections import Counter

counter_lens = Counter(sample_lens)
dict_proportion = { k:v/len(sample_lens) for k,v in counter_lens.items() }

In [17]:
dict_proportion

{3: 0.040229885057471264, 2: 0.14367816091954022, 1: 0.8160919540229885}